---
title: Migrating from MapRerankDocumentsChain
---

[MapRerankDocumentsChain](https://api.python.langchain.com/en/latest/chains/langchain.chains.combine_documents.map_rerank.MapRerankDocumentsChain.html) implements a strategy for analyzing long texts. The strategy is as follows:

- Split a text into smaller documents;
- Map a process to the set of documents, where the process includes generating a score;
- Rank the results by score and return the maximum.

A common process in this scenario is question-answering using pieces of context from a document. Forcing the model to generate score along with its answer helps to select for answers generated only by relevant context.

An [LCEL](/docs/concepts/#langchain-expression-language-lcel) implementation allows for the incorporation of [tool calling](/docs/concepts/#functiontool-calling) and other features for this problem. Below we will go through both `MapRerankDocumentsChain` and a corresponding LCEL implementation on a simple example for illustrative purposes.

## Example

Let's go through an example where we analyze a set of documents. We first generate some simple documents for illustrative purposes:

In [2]:
from langchain_core.documents import Document

documents = [
    Document(page_content="Alice has blue eyes", metadata={"title": "book_chapter_2"}),
    Document(page_content="Bob has brown eyes", metadata={"title": "book_chapter_1"}),
    Document(
        page_content="Charlie has green eyes", metadata={"title": "book_chapter_3"}
    ),
]

### Legacy

Below we show an implementation with `MapRerankDocumentsChain`. We define the prompt template for a question-answering task and instantiate a [LLMChain](https://api.python.langchain.com/en/latest/chains/langchain.chains.llm.LLMChain.html) object for this purpose. We define how documents are formatted into the prompt and ensure consistency among the keys in the various prompts.

In [7]:
from langchain.chains import LLMChain, MapRerankDocumentsChain
from langchain.output_parsers.regex import RegexParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import OpenAI

document_variable_name = "context"
llm = OpenAI()
# The prompt here should take as an input variable the
# `document_variable_name`
# The actual prompt will need to be a lot more complex, this is just
# an example.
prompt_template = (
    "What color are Bob's eyes? "
    "Output both your answer and a score (1-10) of how confident "
    "you are in the format: <Answer>\nScore: <Score>.\n\n"
    "Provide no other commentary.\n\n"
    "Context: {context}"
)
output_parser = RegexParser(
    regex=r"(.*?)\nScore: (.*)",
    output_keys=["answer", "score"],
)
prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["context"],
    output_parser=output_parser,
)
llm_chain = LLMChain(llm=llm, prompt=prompt)
chain = MapRerankDocumentsChain(
    llm_chain=llm_chain,
    document_variable_name=document_variable_name,
    rank_key="score",
    answer_key="answer",
)

In [8]:
response = chain.invoke(documents)
response["output_text"]

/Users/chestercurme/repos/langchain/libs/langchain/langchain/chains/llm.py:369: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


'Brown'

Inspecting the [LangSmith trace](https://smith.langchain.com/public/7a071bd1-0283-4b90-898c-6e4a2b5a0593/r) for the above run, we can see three LLM calls-- one for each document-- and that the scoring mechanism mitigated against hallucinations.

### LCEL

Below we show an LCEL implementation of this process. Note that our template is simplified, as we delegate the formatting instructions to the chat model's tool-calling features via the [.with_structured_output](/docs/how_to/structured_output/) method.

In [9]:
from typing import List

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables.base import RunnableEach
from langchain_openai import ChatOpenAI
from typing_extensions import Annotated, TypedDict


class AnswerWithScore(TypedDict):
    answer: str
    score: Annotated[int, ..., "Score from 1-10."]


llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

prompt_template = "What color are Bob's eyes?\n\n" "Context: {context}"
prompt = ChatPromptTemplate.from_template(prompt_template)


def _format_document(doc):
    return doc.page_content


def _pick_top_ranked(results: List[dict]) -> dict:
    return sorted(results, key=lambda x: -int(x["score"]))[0]


analyze_document_chain = (
    _format_document  # convert from Document object to string
    | prompt  # format into prompt
    | llm.with_structured_output(AnswerWithScore)  # generate structured response
)

chain = (
    RunnableEach(bound=analyze_document_chain)  # run chain on each document
    | _pick_top_ranked  # select top result based on score
)

In [10]:
chain.invoke(documents)

{'answer': 'Bob has brown eyes.', 'score': 10}

Inspecting the [LangSmith trace](https://smith.langchain.com/public/af84ca6d-6ee4-42fe-9134-531ac264a791/r) for the above run, we can see three LLM calls as before. Using the model's tool-calling features have also enabled us to remove the parsing step.

## Next steps

Check out the [LCEL conceptual docs](/docs/concepts/#langchain-expression-language-lcel) for more background information.

See these [how-to guides](/docs/how_to/#qa-with-rag) for more on question-answering tasks with RAG.